# BCI classification

## Import Necessary Libraries

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta
from sklearn.decomposition import PCA
from scipy.fft import fft

## Import libraries for the model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results 
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Selecting Data Columns
* Tag names are loaded in sequential order.
* The process of selecting the required tag names from the tag name list.

In [2]:
# Function to display tag names
def show_column(URL):
    
    # Load tag name data
    df = pd.read_csv(URL)
    
    # Convert to list format
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## Set parameters for displaying tag names
table = 'bci1'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get-tag-names.tql?table={table}'

## Generate tag name list
tag_name = show_column(NAME_URL)

In [4]:
tag_name

['test-s-0',
 'test-s-1',
 'test-s-10',
 'test-s-11',
 'test-s-12',
 'test-s-13',
 'test-s-14',
 'test-s-15',
 'test-s-16',
 'test-s-17',
 'test-s-18',
 'test-s-19',
 'test-s-2',
 'test-s-20',
 'test-s-21',
 'test-s-22',
 'test-s-23',
 'test-s-24',
 'test-s-25',
 'test-s-26',
 'test-s-27',
 'test-s-28',
 'test-s-29',
 'test-s-3',
 'test-s-30',
 'test-s-31',
 'test-s-32',
 'test-s-33',
 'test-s-34',
 'test-s-35',
 'test-s-36',
 'test-s-37',
 'test-s-38',
 'test-s-39',
 'test-s-4',
 'test-s-40',
 'test-s-41',
 'test-s-42',
 'test-s-43',
 'test-s-44',
 'test-s-45',
 'test-s-46',
 'test-s-47',
 'test-s-48',
 'test-s-49',
 'test-s-5',
 'test-s-50',
 'test-s-51',
 'test-s-52',
 'test-s-53',
 'test-s-54',
 'test-s-55',
 'test-s-56',
 'test-s-57',
 'test-s-58',
 'test-s-59',
 'test-s-6',
 'test-s-60',
 'test-s-61',
 'test-s-62',
 'test-s-63',
 'test-s-7',
 'test-s-8',
 'test-s-9',
 'train-s-0',
 'train-s-1',
 'train-s-10',
 'train-s-11',
 'train-s-12',
 'train-s-13',
 'train-s-14',
 'train-s-1

## Converting TAG Name Format
* After checking all the Tag Names from the BCI1 dataset in the previous step, extract only the columns to be used and convert them into parameter format.
* Select all training data.

In [5]:
# Set the desired tag names
tags = tag_name[64:]

# Wrap each item in the list with single quotes and separate with commas
tags_= ",".join(f"'{tag}'" for tag in tags)

# Check the selected tag names
print(tags_)

'train-s-0','train-s-1','train-s-10','train-s-11','train-s-12','train-s-13','train-s-14','train-s-15','train-s-16','train-s-17','train-s-18','train-s-19','train-s-2','train-s-20','train-s-21','train-s-22','train-s-23','train-s-24','train-s-25','train-s-26','train-s-27','train-s-28','train-s-29','train-s-3','train-s-30','train-s-31','train-s-32','train-s-33','train-s-34','train-s-35','train-s-36','train-s-37','train-s-38','train-s-39','train-s-4','train-s-40','train-s-41','train-s-42','train-s-43','train-s-44','train-s-45','train-s-46','train-s-47','train-s-48','train-s-49','train-s-5','train-s-50','train-s-51','train-s-52','train-s-53','train-s-54','train-s-55','train-s-56','train-s-57','train-s-58','train-s-59','train-s-6','train-s-60','train-s-61','train-s-62','train-s-63','train-s-7','train-s-8','train-s-9','train-s-answer'


## Load BCI1 Dataset
* Load the data using the Tag Names.

In [6]:
# Data loading function
def data_load(table, tag_name, name, start_time, end_time, timeformat):
    
    # List to store the results
    result_dfs = []
    
    # Load data
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')

    # Convert to data grouped by the time
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()

    # Separate target values
    df_label = df.iloc[:, -1:].dropna()

    # Remove target column
    df = df.iloc[:, :-1]

    for col_name in tag_name[64:-1]:

        # Set TIME column
        df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')

        # Group by 3-second intervals and count the number of records
        df_counts = df.groupby(df['TIME'].dt.floor('3S')).size().reset_index(name='count')

        # Filter only groups with the most common count
        most_common_count = df_counts['count'].mode()[0]
        filtered_df_counts = df_counts[df_counts['count'] == most_common_count]

        # Convert filtered time values to a list
        filtered_times = filtered_df_counts['TIME'].tolist()

        # Select only the filtered time values from the original DataFrame
        filtered_data = df[df['TIME'].dt.floor('3S').isin(filtered_times)]

        # Group by TIME (rounded to 3-second intervals)
        filtered_data_ = filtered_data.copy()
        filtered_data_['TIME'] = filtered_data_['TIME'].dt.floor('3S')
        grouped = filtered_data_.groupby('TIME')[col_name].apply(list).reset_index()

        # Split the list into individual columns
        result_df = pd.DataFrame(grouped[col_name].tolist())

        # Add result to the list
        result_dfs.append(result_df)
        
    # Initialize the list to store results
    data_list = []
    k = 0

    for k in result_dfs:
        
        # Convert to array format
        data = k.values
        data_list.append(data)

    # Convert the list to a NumPy array
    data_array = np.array(data_list)

    # Reshape to the required format
    # Transform to the shape (number of data, 64, 3000)
    reshaped_array = np.transpose(data_array, (1, 0, 2)) 
    
    # Modify 'train-s-answer'
    df_label.loc[df_label['train-s-answer'] == -1.0, 'train-s-answer'] = 0
    df_label['train-s-answer'] = df_label['train-s-answer'].astype(int)

    return reshaped_array, df_label

In [ ]:
# Data time loading function
def time_data_load(table, name, start_time, end_time, timeformat):
    
    target = 'TIME'
    
    # Load the data 
    df = pd.read_csv(f"http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?target={target}&table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}")
    
    # Create a dummy value column for resampling
    df['value'] = 0
    
    # Perform resampling
    df['TIME'] = pd.to_datetime(df['TIME'])
    df.set_index('TIME', inplace=True)
    # Determine resampling units based on the data and perform resampling
    df = df.resample('3S').mean()
    
    # Remove missing values
    df = df.dropna()
    
    # Remove the dummy value column
    df = df.drop(['value'], axis=1)
    
    return df

In [6]:
# Time update function
# Update start and end times based on batch size
def update_time(time_df, start_time, batch_size):
    
    # Calculate how many data points need to be loaded
    time = batch_size
    
    # Check the index number of the current time
    # If not found, set to the first index as there is no data for the current time
    try:
        index_now = time_df.index.get_loc(start_time)
    except KeyError:
        index_now = 0
    
    # Set the end time for the batch data based on the current time 
    end_time_ = str(time_df.index[index_now + time] + timedelta(seconds=1))
    
    # Set the index number for the next start time
    index_next = index_now + time
    
    # Set the next start time
    next_start_time_ = str(time_df.index[index_next])
    
    # URL encoding
    start_time_ = quote(start_time)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_, index_next

## Data Preprocessing
   * 1 Hanning Window
   * 2 FFT 
   * 3 PCA -> Include in the training process

### 1. Hanning window function setup 

In [9]:
# Hanning window function setup 
def hanning_window(length):
    return 0.5 * (1 - np.cos(2 * np.pi * np.arange(length) / (length - 1)))

### 2. FFT (Fast Fourier Transform) function setup

In [10]:
# FFT transformation function
def change_fft(sample_rate, data):
    # Total number of samples in the signal
    N = sample_rate
    
    # Initialize an array to store FFT results for each channel
    fft_results = np.zeros((data.shape[0], data.shape[1], N // 2 + 1), dtype=float)
    
    # Apply FFT to the entire dataset
    for i in range(data.shape[0]):  # For each sample
        for j in range(data.shape[1]):  # For each channel
            yf = fft(data[i, j], n=N)  # Calculate FFT
            # Compute the absolute value of the FFT result and normalize (only the meaningful part)
            fft_results[i, j] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    return fft_results

## Model Configuration
* Using ResNet1d model.

In [11]:
## ResNet 1D Model Setup 
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Identity mapping
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64 

        
        self.conv1 = nn.Conv1d(64, 64, kernel_size=7, stride=2, padding=3) 
        self.bn1 = nn.BatchNorm1d(64)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1])
        self.layer3 = self._make_layer(block, 256, layers[2])
        self.layer4 = self._make_layer(block, 512, layers[3])
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks):
        layers = []
        layers.append(block(self.in_channels, out_channels))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [12]:
# Model configuration parameters
# Learning rate
learning_rate = 0.01

# Model configuration
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=2).to(device)

# Configure loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check the model architecture
print(model)

ResNet(
  (conv1): Conv1d(64, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequ

## Model Training
* Proceed by loading only the necessary batch size of data for training.
* Save the model with the highest F1 score based on the validation data during training.

In [13]:
# Model training function
def train(table, tag_name, name, timeformat, model, batch_size_train, batch_size_valid, epochs, time_df_train, time_df_valid, pca, sample_rate):
    
    # Initialize training loss & accuracy
    train_loss = []
    train_acc = []
    
    # Initialize best F1 Score value
    best_f1= 0

    # Start model training
    for epoch in epochs:
        model.train()
        
        running_loss = 0.0
        total_step = 0
        correct = 0
        total=0
        
        # Set initial start time
        start_time_ = str(time_df_train.index[0])
        
        # Set end time
        end_time_train = str(time_df_train.index[-1])
        
        # Use a while loop to call data
        while start_time_ < end_time_train:
            
            # Set the time for loading data based on the batch size 
            start_time_, end_time_, next_start_time_, index_next= update_time(time_df_train, start_time_, batch_size_train)
        
            # Load batch data 
            data, label = data_load(table, tag_name, name, start_time_, end_time_, timeformat)
            
            # Apply Hanning window
            data = data * hanning_window(sample_rate)
            
            # Apply FFT
            data = change_fft(sample_rate, data)
            
            # Apply PCA
            # Reshape to 2D
            data_ = data.reshape(-1, data.shape[2])
            data_ = pca.fit_transform(data_)
            # Reshape back to original 3D format
            data = data_.reshape(data.shape[0], data.shape[1], -1)
            
            # Print if the loaded data is empty 
            if len(data) == 0:
                print("No data available.")
            
            # Input the data into the model when it accumulates to the batch size
            if len(data) == batch_size_train:
                
                # Check total batch count
                total_step = total_step + 1
                
                # Convert data to numpy arrays
                input_data = np.array(data)
                input_target = np.array(label)[:-1]

                # Convert data to Tensor
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()
                input_target = torch.tensor(input_target, dtype=torch.float32).to(device).long().squeeze()
                
                # Optimize the optimizer
                optimizer.zero_grad()
                
                # Input to the model
                outputs = model(input_data)
                
                # Calculate loss
                loss = criterion(outputs, input_target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
                # Set label predictions
                _,pred = torch.max(outputs, dim=1)
                correct += torch.sum(pred==input_target).item()
                total += input_target.size(0)

                # Reset batch data
                data = []
                
            # Set the next start time   
            start_time_ = unquote(next_start_time_)
            
            # Prevent fetching beyond the last time
            if index_next + batch_size_train >= len(time_df_train):
                break
            
        train_acc.append(100 * correct / total)
        train_loss.append(running_loss/total_step)
        print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')
        
        # Perform validation at the end of each epoch and save the model with the best performance
        with torch.no_grad():
            
            model.eval()
            
            preds_v = []
            targets_v = []
                
            # Set initial start time
            start_time_v = str(time_df_valid.index[0])
            
            # Set end time
            end_time_valid = str(time_df_valid.index[-1])
            
            # Use a while loop to call data 
            while start_time_v < end_time_valid:
                
                # Set the time for loading data based on the batch size
                start_time_v, end_time_v, next_start_time_v, index_next_v = update_time(time_df_valid, start_time_v, batch_size_valid)
                
                # Load batch data 
                data_v, label_v = data_load(table, tag_name, name, start_time_v, end_time_v, timeformat)
                
                # Apply Hanning window
                data_v = data_v * hanning_window(sample_rate)
                
                # Apply FFT
                data_v = change_fft(sample_rate, data_v)
                
                # Apply PCA
                # Reshape to 2D
                data_ = data_v.reshape(-1, data_v.shape[2])
                data_ = pca.fit_transform(data_)
                # Reshape back to original 3D format
                data_v = data_.reshape(data_v.shape[0], data_v.shape[1], -1)
                
                # Print if the loaded data is empty
                if len(data_v) == 0:
                    print("No data available.")
                
                # Input the data into the model when it accumulates to the batch size
                if len(data_v) == batch_size_valid:
                    
                    # Convert data to numpy arrays
                    input_data_v = np.array(data_v)
                    input_target_v = np.array(label_v)[:-1]

                    # Convert data to Tensor
                    input_data_v = torch.tensor(input_data_v, dtype=torch.float32).to(device).float()
                    input_target_v = torch.tensor(input_target_v, dtype=torch.float32).to(device).long().squeeze()
                    
                    # Input to the model
                    outputs_v = model(input_data_v)
                    
                    # Set label predictions 
                    _,pred_v = torch.max(outputs_v, dim=1)
                    target_v = input_target_v.view_as(pred_v)
    
                    preds_v.append(pred_v)
                    targets_v.append(target_v)
                    
                    # Reset batch data
                    data_v = []
                    
                # Set the next start time    
                start_time_v = unquote(next_start_time_v)
                
                # Prevent fetching beyond the last time
                if index_next_v + batch_size_valid >= len(time_df_valid):
                    break
                    
            # Combine predictions and labels collected from all batches
            preds_v = torch.cat(preds_v).detach().cpu().numpy()
            targets_v = torch.cat(targets_v).detach().cpu().numpy()
            
            f1score = f1_score(targets_v, preds_v,  average='macro')
            if best_f1 < f1score:
                best_f1 = f1score
                # Save the best model 
                with open("./result/BCI1_ResN et1d_New_Batch.txt", "a") as text_file:
                    print('epoch=====',epoch, file=text_file)
                    print(classification_report(targets_v, preds_v, digits=4), file=text_file)
                torch.save(model, f'./result/BCI1_ResNet1d_New_Batch.pt') 
            epochs.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")
                    
    return model

In [14]:
########################################### Training Parameter Settings ################################################
# Set tag table name
table = 'bci1'
# Set tag name
name = quote(tags_, safe=":/")
# Set time format
timeformat = quote('2006-01-02 15:04:05.000000')
# Set the start time for the train data
start_time_train = '2024-01-01 00:00:00'
# Set the end time for the train data
end_time_train = '2024-01-01 03:41:00'
# Set train batch size
batch_size_train = 16
# Set number of epochs
epochs = trange(100, desc='training')
# Set sample rate
sample_rate = 3000
# Set PCA
# Select principal components that explain 95% of the variance
pca = PCA(n_components=14)
# Load training time list 
time_df_train = time_data_load(table, name, quote(start_time_train), quote(end_time_train), timeformat)

########################################### validation Parameter Settings ################################################
# Set the start time for the validation data
start_time_valid = '2024-01-01 03:42:00'
# Set the end time for the validation data
end_time_valid = '2024-01-01 04:09:00'
# Set vaild batch size
batch_size_valid = 16
# Load validation time list
time_df_valid = time_data_load(table, name, quote(start_time_valid), quote(end_time_valid), timeformat)

########################################### Proceed with training ################################################
train(table, tag_name, name, timeformat, model, batch_size_train, batch_size_valid, epochs, time_df_train, time_df_valid, pca, sample_rate)


training:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.2080353131661048, train acc: 47.5962

train loss: 1.1204177118264713, train acc: 49.0385


c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train loss: 1.0725246285780883, train acc: 53.3654

train loss: 0.9887269127827425, train acc: 50.4808

train loss: 0.949872727577503, train acc: 50.0000

train loss: 0.9121195398844207, train acc: 51.9231

train loss: 0.8853872693501987, train acc: 53.3654

train loss: 0.8643282795181642, train acc: 52.8846

train loss: 0.8463760217030843, train acc: 51.9231

train loss: 0.8322952550191145, train acc: 57.2115

train loss: 0.8204017089797067, train acc: 55.2885

train loss: 0.8087945209863858, train acc: 62.0192

train loss: 0.7943512659806472, train acc: 66.3462

train loss: 0.7745996657963637, train acc: 76.9231

train loss: 0.7561487672802729, train acc: 83.1731

train loss: 0.7251674725602453, train acc: 89.9038

train loss: 0.6985537067257981, train acc: 91.3462

train loss: 0.669987006733815, train acc: 93.2692

train loss: 0.6403646057826065, train acc: 97.1154

train loss: 0.6117753831777148, train acc: 97.5962

train loss: 0.5850418515865019, train acc: 98.5577

train loss: 0

ResNet(
  (conv1): Conv1d(64, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequ

## Model Testing

In [16]:
# Model testing function  
def test(table, tag_name, name, timeformat, model, batch_size, time_df_test, pca, sample_rate):
    
            with torch.no_grad():
                
                model.eval()
                
                # Initial settings 
                preds_t = []
                targets_t = []
                    
                # Set the initial start time
                start_time_t = str(time_df_test.index[0])
                
                # Set the end time
                end_time_test = str(time_df_test.index[-1])
                
                # Use a while loop to call data  
                while start_time_t < end_time_test:
                    
                    # Set the time for loading data based on the batch size
                    start_time_t, end_time_t, next_start_time_t, index_next_t = update_time(time_df_test, start_time_t, batch_size)
                    
                    # Load batch data
                    data_t, label_t = data_load(table, tag_name, name, start_time_t, end_time_t, timeformat)
                    
                    # Apply Hanning window
                    data_t = data_t * hanning_window(sample_rate)
                    
                    # Apply FFT
                    data_t = change_fft(sample_rate, data_t)
                    
                    # Apply PCA
                    # Reshape to 2D
                    data_ = data_t.reshape(-1, data_t.shape[2])
                    data_ = pca.fit_transform(data_)
                    # Reshape back to original 3D format
                    data_t = data_.reshape(data_t.shape[0], data_t.shape[1], -1)
                    
                    # Print if the loaded data is empty
                    if len(data_t) == 0:
                        print("No data available.")
                    
                    # Input the data into the model when it accumulates to the batch size
                    if len(data_t) == batch_size:
                        
                        # Convert data to numpy arrays
                        input_data_t = np.array(data_t)
                        input_target_t = np.array(label_t)[:-1]

                        # Convert data to Tensor
                        input_data_t = torch.tensor(input_data_t, dtype=torch.float32).to(device).float()
                        input_target_t = torch.tensor(input_target_t, dtype=torch.float32).to(device).long().squeeze()
                        
                        # Input to the model
                        outputs_t = model(input_data_t)
                        
                        # Set label predictions
                        _,pred_t = torch.max(outputs_t, dim=1)
                        target_t = input_target_t.view_as(pred_t)
        
                        preds_t.append(pred_t)
                        targets_t.append(target_t)
                        
                        # Reset batch data
                        data_t = []
                        
                    # Set the next start time   
                    start_time_t = unquote(next_start_time_t)
                    
                    # Prevent fetching beyond the last time
                    if index_next_t + batch_size + 1 >= len(time_df_test):
                        break
                        
                # Combine predictions and labels collected from all batches
                preds_t = torch.cat(preds_t).detach().cpu().numpy()
                targets_t = torch.cat(targets_t).detach().cpu().numpy()
                    
            return targets_t, preds_t

In [19]:
########################################### Test Parameter Settings ################################################
# Load the best model
model_ = torch.load(f'./result/BCI1_ResNet1d_New_Batch.pt') 
# Set the start time for the test data
start_time_test = '2024-01-01 04:10:00'
# Set the end time for the test data
end_time_test = '2024-01-01 04:37:01'
# Set the test batch size
batch_size_test = 4
# Load the test time list
time_df_test = time_data_load(table, name, quote(start_time_test), quote(end_time_test), timeformat)

######################################## Proceed with testing #############################################
targets_t, preds_t = test(table, tag_name, name, timeformat, model_, batch_size_test, time_df_test, pca, sample_rate)

## Model Performance Evaluation

In [20]:
print(classification_report(targets_t, preds_t))

              precision    recall  f1-score   support

           0       1.00      0.64      0.78        14
           1       0.67      1.00      0.80        10

    accuracy                           0.79        24
   macro avg       0.83      0.82      0.79        24
weighted avg       0.86      0.79      0.79        24

